<a href="https://colab.research.google.com/github/ahmedmedoX/Brain-Tumor-Classification/blob/main/Brain_Tumor_Classification_V1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Libraries**

In [1]:
import os
import cv2
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics

## Calculate Third Moment (Skewness)

In [2]:
def calculate_third_moment(image):
    pixels = np.array(image).flatten()
    mean = np.mean(pixels)
    std_dev = np.std(pixels)
    third_moment = np.mean(((pixels - mean) / std_dev) ** 3)
    return third_moment

## Calculate Contrast

In [3]:
def calculate_average_contrast(image):
    contrast = np.abs(image - np.mean(image))
    average_contrast = np.mean(contrast)
    return average_contrast

## Calculate Smoothness

In [4]:
def calculate_smoothness(image):
    dx = np.abs(np.diff(image, axis=1))
    dy = np.abs(np.diff(image, axis=0))
    smoothness = (np.mean(dx) + np.mean(dy)) / 2
    return smoothness

## Calculate Entropy

In [5]:
def calculate_entropy(image):
    histogram=cv2.calcHist([image],[0],None,[256],[0,256])
    total_pixels = image.shape[0] * image.shape[1]
    entropy = 0
    for count in histogram:
        probability = count / total_pixels
        if probability > 0:
            entropy += probability * math.log2(probability)
    entropy *= -1
    return entropy

## Calculate Intensity

In [6]:
def calculate_intensity(image):
    pixels = np.array(image)
    intensity = np.mean(pixels)
    return intensity

## Segmentation

In [7]:
def Binary_Threshold(image):
  image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  B_Threshold=cv2.threshold(image_gray,125,255,cv2.THRESH_BINARY)
  return B_Threshold

## Read Images

In [8]:
Images_No_Path = '/content/No'
Images_Yes_Path = '/content/Yes'

Images_No=[]
Images_Yes=[]

File_Names= os.listdir(Images_No_Path)
for i in File_Names:
  Images_No.append(cv2.imread(Images_No_Path+'/'+i))

File_Names= os.listdir(Images_Yes_Path)
for i in File_Names:
  Images_Yes.append(cv2.imread(Images_Yes_Path+'/'+i))

## Store Features

In [9]:
Features={'Entropy':[],
         'Third Moment':[],
         'Intensity':[],
         'Contrast':[],
         'Smoothness':[],
         'Class':[]} #Yes or No

Processed_Images_No=[]
Processed_Images_Yes=[]

for i in Images_No:
  Processed_Images_No.append(Binary_Threshold(i))
for i in Images_Yes:
  Processed_Images_Yes.append(Binary_Threshold(i))

for i in Processed_Images_No:
  Features['Entropy'].append(calculate_entropy(i[1]))
  Features['Third Moment'].append(calculate_third_moment(i[1]))
  Features['Intensity'].append(calculate_intensity(i[1]))
  Features['Contrast'].append(calculate_average_contrast(i[1]))
  Features['Smoothness'].append(calculate_smoothness(i[1]))
  Features['Class'].append(0)

for i in Processed_Images_Yes:
  Features['Entropy'].append(calculate_entropy(i[1]))
  Features['Third Moment'].append(calculate_third_moment(i[1]))
  Features['Intensity'].append(calculate_intensity(i[1]))
  Features['Contrast'].append(calculate_average_contrast(i[1]))
  Features['Smoothness'].append(calculate_smoothness(i[1]))
  Features['Class'].append(1)

dataset = pd.DataFrame(Features)
dataset.to_csv('/content/Features.csv', index=False)

<ipython-input-5-3d26f5294651>:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  entropy += probability * math.log2(probability)


## Train Model

In [10]:
X = dataset.drop('Class', axis=1)
y = dataset['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [11]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

DecisionTreeClassifier()

## Evaluate Model

In [18]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

[[15 17]
 [ 8 36]]
Accuracy: 0.6710526315789473
